# 13章　ナイーブベイズ




## 13.1 非常に単純なスパムフィルタ
メッセージがスパムである事象S、メッセージにビットコインという単語が含まれる事象Bで表すと
ベイズの定理で、メッセージにビットコインという単語が含まれていた場合、
そのメッセージがスパムである確率は

```
P(S | B) = [P(B | S)P(S)] / [P(B | S)P(S)+P(B | ¬S)P(¬S)]

```
事象Bにおける事象Sの条件月確率。
分子はメッセージがスパムであり、ビットコインという単語を含んでいる確率を表し、
分母はメッセージにビットコインという単語を含んでいる確率を表している。
（P(¬S)はSが発生しない確率）

スパムであることがわかっているメッセージとスパムでないことがわかっているメッセージが大量にあるなら
P(B | S)とP(B | ¬S)を計算するのは簡単（スパムメッセージのうちビットコインという単語を含む確率と、スパムではないメッセージがビットコインという単語を含む確率）

```
P (S | B ) = P (B | S ) / [P (B | S ) + P (B | ¬ S )]
スパムメッセージのうちビットコインという単語が含まれている確率（=0.5) /(その0.5 +  スパムでないメッセージにビットコインが含まれる確率 0.01)
```

0.5/(0.5+0.01) = 98%


## 13.2 より高度なスパムフィルター

語彙リストW1,...,Wnがあるとして
メッセージが単語Wiを含む確率をXiとすると
スパムメッセージがi番目の単語を含む確率P(Xi | S)とスパムではないメッセージがi番目の単語を含む確率P(Xi | ¬ S )の推定が得られているとする。

各単語の有無は別の単語の有無とは独立であると仮定する点がナイーブベーズの肝、とのこと

スパムメッセージに「ビットコイン」と「ロレックス」が両方書かれている確率は

P (X1=1, X2=1 | S ) = P (X1=1 | S ) P (X2=1 | S ) = 0.5×0.5 = 0.25



## 13.3 実装

ここまでの式を用いていスパムフィルターを実装する

In [37]:
# メッセージを個々の単語に分割する簡単な関数

from typing import Set
import re

def tokenize(text: str) -> Set[str]:
    text = text.lower()
    # アルファベットと小文字で構成される文字列（基本的に英語の場合単語）を見つけ収集する
    all_words = re.findall("[a-z0-9]+", text)
    return set(all_words)

assert tokenize("Data Science is science") == {"data", "science", "is"}


In [38]:
# 学習用データの型を定義

from typing import NamedTuple

# NamedTupleなコンテナの定義
# Message(text="hoge", is_spam=True) のように定義できる
class Message(NamedTuple):
    text: str
    is_spam: bool
        

In [52]:
# 分類器の実装

# コンストラクタではカウント、ラベルを追跡するため、クラスを定義する:
# 擬似カウントk, 空の単語の集合。
# スパムメッセージおよびハムメッセージで各単語が表示される頻度のカウンター
# 学習したスパムおよびハムメッセージ数のカウント

from typing import List, Tuple, Dict, Iterable
import math
from collections import defaultdict

class NaiveBayesClassifier:
    def __init__(self, k: float = 0.5) -> None:
        # k:スムージングのための変数
        self.k = k
        self.tokens: Set[str] = set()
        # 単語のインクリメントなカウントなのでdefaultdict
        self.token_spam_counts: Dict[str, int] = defaultdict(int)
        self.token_ham_counts: Dict[str, int] = defaultdict(int)
        # 二重に初期化しているっぽい。あまりこの書き方しないかも
        self.spam_messages = self.ham_messages = 0
        
#　spam_messagesとham_messagesのカウントを行い学習していく
    # MessageクラスのインスタンスのIterable型が引数
    def train(self, messages: Iterable[Message]) -> None:
        for message in messages:
            # is_spamのbool値、初期値から実際の値にどう代入されているの？？
            if message.is_spam:
                self.spam_messages += 1
            else:
                self.ham_messages += 1
            
            for token in tokenize(message.text):
                # 分類器で使うtokensにtokenを追加
                self.tokens.add(token)
                if message.is_spam:
                    self.token_spam_counts[token] += 1
                else:
                    self.token_ham_counts[token] += 1
                    
# 最終的なP(スパム | 単語)を予測するために必要な、P(単語 | スパム)とP(単語 | ハム)を生成する
    def _probabilities(self, token: str) -> Tuple[float, float]:
        spam = self.token_spam_counts[token]
        ham = self.token_ham_counts[token]
        p_token_spam = (spam + self.k) / (self.spam_messages + 2 * self.k)
        p_token_ham = (ham + self.k) / (self.ham_messages + 2 * self.k)
        return p_token_spam, p_token_ham
    
# 最終的な確率。対数確率を合計する（確率の乗算は小数点の桁数が膨大になるため対数をとると扱いやすい場合がある）
    def predict(self, text: str) -> float:
        # 単語セット
        text_tokens = tokenize(text)
        log_prob_if_spam = log_prob_if_ham = 0.0
        
        for token in self.tokens:
            prob_if_spam, prob_if_ham = self._probabilities(token)
            
            if token in text_tokens:
                log_prob_if_spam += math.log(prob_if_spam)
                log_prob_if_ham += math.log(prob_if_ham)
            else:
                log_prob_if_spam += math.log(1.0 - prob_if_spam)
                log_prob_if_ham += math.log(1.0 - prob_if_ham)
        prob_if_spam = math.exp(log_prob_if_spam)
        prob_if_ham = math.exp(log_prob_if_ham)
        return prob_if_spam / (prob_if_spam + prob_if_ham)


## 13.4 モデルの検証

messagesのテストデータを作成し、ユニットテストを行いモデルが機能することを確認する。クラス定義どおり、messagesはテキスト本文と**is_spamのbool値を持つ**Messageインスタンスのリスト。

In [53]:
messages = [Message("spam rules", is_spam=True), 
            Message("ham rules", is_spam=False), 
            Message("hello ham", is_spam=False)]

model = NaiveBayesClassifier(k=0.5)
model.train(messages)

In [54]:
# カウントのテスト OK
assert model.tokens == {"spam", "ham", "rules", "hello"}
assert model.spam_messages == 1
assert model.ham_messages == 2
assert model.token_spam_counts == {"spam": 1, "rules": 1}
assert model.token_ham_counts == {"ham": 2, "rules": 1, "hello": 1}

In [55]:
# 予測のテスト
# ロジックを手入力で追試したものと"text"をmessageしとした場合の予測を比較
# 予測される確率はおおよそ0.835となり手計算と一致する

text = "hello spam"

probs_if_spam = [
(1 + 0.5) / (1 + 2 * 0.5),
1 - (0 + 0.5) / (1 + 2 * 0.5), 1 - (1 + 0.5) / (1 + 2 * 0.5), (0 + 0.5) / (1 + 2 * 0.5)
]

probs_if_ham = [
(0 + 0.5) / (2 + 2 * 0.5),
1 - (2 + 0.5) / (2 + 2 * 0.5), 1 - (1 + 0.5) / (2 + 2 * 0.5), (1 + 0.5) / (2 + 2 * 0.5),
]

p_if_spam = math.exp(sum(math.log(p) for p in probs_if_spam)) 
p_if_ham = math.exp(sum(math.log(p) for p in probs_if_ham))

print(p_if_spam, p_if_ham)
print(p_if_spam / (p_if_spam + p_if_ham))
print(model.predict(text))

assert model.predict(text) == p_if_spam / (p_if_spam + p_if_ham)

0.03515625 0.006944444444444444
0.8350515463917525
0.8350515463917525


## 13.5 モデルの適用